# Claude vs GPT

**Improvements**
- Pause the convo and continue without restarting.
- Add paramter to disable streaming in the backend as it is unecessary for our gradio function.
- Choose the topic ✅

In [1]:
import sys
sys.path.append("..")

from src.chatbot import Socrates, Eris
from src.test_chatbot import TestSocrates, TestEris

from IPython.display import Markdown, display
import gradio as gr
import time
import random

c:\Users\becaye.balde\Desktop\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Debate: Claude VS GPT

### Functions

#### Tests

In [10]:
# Regular generator (not async) that yields partial message updates
def stream_message(history, message, is_socrates=True, delay=0.025):
    partial = ""
    for char in message:
        partial += char
        # Update the last message in the chat history
        if is_socrates:
            if history and history[-1][1] is None:
                history[-1] = (partial, None)
            else:
                history.append((partial, None))
        else:
            if history and history[-1][0] is None:
                history[-1] = (None, partial)
            else:
                history.append((None, partial))
        time.sleep(delay)
        yield history


# Main conversation loop
def test_adversarial_chat(n_rounds):
    socrates = TestSocrates()
    eris = TestEris()
    history = []

    time.sleep(random.uniform(1.0, 1.4))
    soc_reply = socrates.ask("What is the nature of justice?")
    for update in stream_message(history, soc_reply, is_socrates=True):
        yield update

    for i in range(n_rounds):
        time.sleep(random.uniform(0.6, 1.2))
        if i % 2 == 0:
            eris.history = socrates.history
            eris_reply = eris.ask(soc_reply)
            for update in stream_message(history, eris_reply, is_socrates=False):
                yield update
        else:
            socrates.history = eris.history
            soc_reply = socrates.ask(eris_reply)
            for update in stream_message(history, soc_reply, is_socrates=True):
                yield update

### Go

In [ ]:
def clean_message(message):
    """
    Replace double newlines with single newline.
    Useful because Claude like new paragraphs but we lack space.
    """
    return message.replace("\n\n", "\n").strip()


def stream_message(history, message, is_socrates=True, delay=0.01):
    """
    Generator that streams message updates character by character.

    Args:
        history (list): Current chat history.
        message (str): Message to stream.
        is_socrates (bool): True if Socrates is speaking, False if Eris.
        delay (float): Delay between character updates.
    Yields:
        Generator yielding updated chat history.
    """
    partial = ""
    for char in message:
        partial += char
        if is_socrates:
            # Socrates appears on user side
            if history and history[-1]["role"] == "user":
                history[-1]["content"] = partial
            else:
                history.append({"role": "user", "content": partial})
        else:
            # Eris appears on assistant side
            if history and history[-1]["role"] == "assistant":
                history[-1]["content"] = partial
            else:
                history.append({"role": "assistant", "content": partial})
        time.sleep(delay)

        # We use yield to create a streaming effect
        yield history


# Main conversation loop
def adversarial_chat(n_rounds, topic):
    """
    Main conversation loop for Socrates and Eris, streaming updates.
    Args:
        n_rounds (int): Number of rounds of conversation.
        topic (str): Topic for the conversation.
    Yields:
        Generator yielding chat history updates.
    """
    # Initialize Socrates and Eris chatbots
    socrates = Socrates()
    eris = Eris()
    history = []

    # Pause to simulate thinking time
    time.sleep(random.uniform(0.3, 1))

    # Start the conversation with Socrates
    soc_reply = socrates.ask(topic)
    soc_reply = clean_message(soc_reply)
    for update in stream_message(history, soc_reply, is_socrates=True):
        yield update

    # Now alternate turns between Socrates and Eris
    for i in range(n_rounds-1):
        time.sleep(random.uniform(0.6, 1))
        
        # Even rounds: Eris responds, odd rounds: Socrates responds
        if i % 2 == 0:
            eris.history = socrates.history
            eris_reply = eris.ask(soc_reply)
            eris_reply = clean_message(eris_reply)
            for update in stream_message(history, eris_reply, is_socrates=False):
                yield update
        else:
            socrates.history = eris.history
            soc_reply = socrates.ask(eris_reply)
            soc_reply = clean_message(soc_reply)
            for update in stream_message(history, soc_reply, is_socrates=True):
                yield update

### GUI

In [3]:
# Gradio UI (edit only input and .click line)
with gr.Blocks() as demo:
    gr.Markdown("## ⚔️ Socrates vs. Eris")
    gr.Markdown(
        "Dive into a battle of minds where **Socrates**, the calm philosopher, asks piercing questions "
        "to uncover truth — and **Eris**, the goddess of discord, responds with sarcastic wit and playful contradiction.  \n"
        "Choose a topic and watch them clash in a brief, animated debate."
    )

    with gr.Row():
        rounds = gr.Number(label="Rounds", value=4, precision=0)
        topic_input = gr.Textbox(label="Debate Topic", placeholder="e.g. What is freedom?")
        start_btn = gr.Button("Start Debate")

    # chatbox = gr.Chatbot(label="Debate Log")
    chatbox = gr.Chatbot(label="Let the battle begin!", type="messages", height=1000)

    def reset_chat():
        return []

    start_btn.click(fn=reset_chat, outputs=chatbox)
    start_btn.click(fn=adversarial_chat,
                    inputs=[rounds, topic_input],
                    outputs=chatbox)

In [ ]:
demo.launch()
# demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Indeed, reason is the supreme tool for human progress and understanding. Through rational thought, we can overcome superstition, emotional impulse, and destructive passions. Reason allows us to analyze problems objectively, develop scientific knowledge, create just social systems, and make ethical decisions that benefit the collective good. Without reason, humanity would be lost in darkness, prey to ignorance and base instincts.

Oh, how adorable! Do you really believe that cold, dispassionate logic can guide us through the chaos of human nature? Reason alone ignores the messy, beautiful emotions that define us—love, passion, even chaos. Without them, we're just soulless machines, not truly human. Reason isn’t a beacon; it’s a prison. Emotions fuel creativity, rebellion, and progress—things reason can never invent or inspire!

Reason is not a cold, detached process, but a powerful method of understanding that integrates emotion with logic. Emotions without reason are volatile and destructive, while reason tempers and directs emotional energy toward constructive ends. Scientific discoveries, human rights, and social progress have all emerged from rational thought. Pure emotion leads to impulsivity, conflict, and suffering. Reason allows us to transcend our immediate impulses and create meaningful, sustainable solutions that benefit humanity as a whole.

Oh, how noble of you to rebrand reason as emotionally intelligent. But let’s not pretend that reason is some infallible, compassionate guide when history shows us it often justifies brutality and tyranny under ideological masks. Rationality without conscience is just a weapon—using the guise of progress to oppress or destroy. Humans aren’t machines; they’re unpredictable chaos wrapped in emotion. Reason might be useful, but it’s just as likely to ruin everything if wielded without the discord and passion that truly make us alive.